In [2]:
import sys
import requests
import pandas as pd 
import math

sell_SL_diff_7d = []
sell_SL_diff_1d = []
buy_SL_diff_7d = []
buy_SL_diff_1d = []

sell_SL_pct_diff_7d = []
sell_SL_pct_diff_1d = []

sell_sold_5pct_1d = []
buy_sold_5pct_1d = []
profit_est = []

url_items_tp_data = (   'https://api.datawars2.ie/gw2/v1/items/json?fields='
                        'id,name,buy_price,sell_price,roi,profit,'
                        '7d_buy_listed,7d_buy_sold,7d_sell_listed,7d_sell_sold,'
                        '1d_buy_listed,1d_buy_sold,1d_sell_listed,1d_sell_sold')


result = requests.get(url_items_tp_data)

if(result.status_code == 200):
    data = result.json()
    df = pd.json_normalize(data)

else:
	sys.exit("Status code not 200")


for i, row in df.iterrows():

    sell_SL_diff_7d.append(row['7d_sell_sold'] - row['7d_sell_listed'])
    sell_SL_diff_1d.append(row['1d_sell_sold'] - row['1d_sell_listed'])
    buy_SL_diff_7d.append(row['7d_buy_sold'] - row['7d_buy_listed'])
    buy_SL_diff_1d.append(row['1d_buy_sold'] - row['1d_buy_listed'])
    
    # Calculate percetage difference with sign for 7 days
    if(not math.isnan(row['7d_sell_sold']) and row['7d_sell_sold'] != 0 and row['7d_sell_listed'] != 0):
        sell_SL_pct_diff_7d.append(((row['7d_sell_sold'] - row['7d_sell_listed']) / ((row['7d_sell_sold'] + row['7d_sell_listed']) / 2)) * 100)
    else:
        sell_SL_pct_diff_7d.append(float('nan'))

    # Calculate percetage difference with sign for 1 day
    if(not math.isnan(row['1d_sell_sold']) and row['1d_sell_sold'] != 0 and row['1d_sell_listed'] != 0):
        sell_SL_pct_diff_1d.append(((row['1d_sell_sold'] - row['1d_sell_listed']) / ((row['1d_sell_sold'] + row['1d_sell_listed']) / 2)) * 100)
    else:
        sell_SL_pct_diff_1d.append(float('nan'))

    if(not math.isnan(row['1d_sell_sold'])):
        # Calculate velocity of item
        buy_sold_5pct_1d.append(int(row['1d_buy_sold'] * 0.05))
        sell_sold_5pct_1d.append(int(row['1d_sell_sold'] * 0.05))

        # Calculate estimated profit based on velocity
        if(sell_sold_5pct_1d[-1] <= buy_sold_5pct_1d[-1]):
            profit_est.append(row['profit'] * sell_sold_5pct_1d[-1])
        else:
            profit_est.append(row['profit'] * buy_sold_5pct_1d[-1])

    else:
        buy_sold_5pct_1d.append(float('nan'))
        sell_sold_5pct_1d.append(float('nan'))
        profit_est.append(float('nan'))

df['sell_SL_diff_7d'] = sell_SL_diff_7d
df['sell_SL_diff_1d'] = sell_SL_diff_1d
df['buy_SL_diff_7d'] = buy_SL_diff_7d
df['buy_SL_diff_1d'] = buy_SL_diff_1d

df['sell_SL_pct_diff_7d'] = sell_SL_pct_diff_7d
df['sell_SL_pct_diff_1d'] = sell_SL_pct_diff_1d

df['bs_speed'] = buy_sold_5pct_1d
df['ss_speed'] = sell_sold_5pct_1d
df['profit_est'] = profit_est

df['roi'] = df['roi'] * 100

# df.round()
# df.round({"roi":2})

df = df.round(decimals=0).astype(object)
df = df.convert_dtypes()

# df.dtypes

NameError: name 'requests' is not defined

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df[[	
		'name',
		'sell_price',
		'buy_price',
		'profit',
		'profit_est',
		'bs_speed',
		'ss_speed',
		'roi',

		'1d_buy_sold',
		'buy_SL_diff_1d',

		'7d_sell_sold',
		# 'sell_SL_diff_7d',
		'sell_SL_pct_diff_7d',

		'1d_sell_sold',
		# 'sell_SL_diff_1d',
		'sell_SL_pct_diff_1d'

		]]

	.loc	[
		(df['profit'] > 20) &
		(df['roi'] > 10) &

		# (df['1d_sell_sold'] > 1) &
		(df['1d_sell_sold'] > 30) &
		# (df['1d_sell_sold'] > 100) &

		# (df['1d_buy_sold'] > 1) &
		(df['1d_buy_sold'] > 10) &
		# (df['1d_buy_sold'] > 50) &

		(df['sell_SL_pct_diff_1d'] >= -10) 

		# (df['sell_SL_diff_7d'] > 0) &
		# (df['sell_SL_diff_1d'] > 0) #&

		# (df['buy_SL_diff_7d'] > 0) &
		# (df['buy_SL_diff_1d'] > -10) #&
		]

	# .sort_values(by='profit_est', ascending=False)
	.sort_values(by='profit', ascending=False)
	# .sort_values(by='ss_speed', ascending=False)

	# .head(30)

	.style
		.hide_index()
		.bar(subset=[	#'buy_SL_diff_7d',
				'buy_SL_diff_1d',
				'sell_SL_pct_diff_7d',
				'sell_SL_pct_diff_1d'
				], align='zero', color=['#d65f5f', '#5fba7d'])
		.set_properties(**{'background-color': '#7b838f'}, subset=['1d_buy_sold', 'buy_SL_diff_1d'])
		.set_properties(**{'background-color': '#8ba6a0'}, subset=['7d_sell_sold', 'sell_SL_pct_diff_7d'])
		.set_properties(**{'background-color': '#7b838f'}, subset=['1d_sell_sold', 'sell_SL_pct_diff_1d'])
		.set_properties(**{'background-color': '#b8a73b'}, subset=['profit_est'])
	# .shape[0]
	)

name,sell_price,buy_price,profit,profit_est,bs_speed,ss_speed,roi,1d_buy_sold,buy_SL_diff_1d,7d_sell_sold,sell_SL_pct_diff_7d,1d_sell_sold,sell_SL_pct_diff_1d
Carbonized Mithrillium Ingot,35787,26216,4203,21015,5,20,16,116,-656,2286,-39,402,-10
Carrion Noble Coat,8800,3630,3850,26950,36,7,106,729,-612,222,1,158,3
Rabid Pearl Handcannon,9591,4595,3557,14228,4,6,77,84,-26,252,-21,123,89
Superior Rune of the Water,26378,20170,2251,13506,6,6,11,123,-16,983,-19,130,30
Vial of Magic,9482,6585,1475,2950,2,8,22,55,-261,1029,-12,167,82
Masterwork Black Lion Dye Canister—Green,6899,4777,1087,9783,13,9,23,274,-1676,822,-22,180,21
Bowl of Spiced Red Lentil Stew,1655,546,861,6888,8,8,158,176,173,194,76,172,189
Writ of Masterful Strength,6092,4505,673,18844,28,36,15,561,273,3087,-14,739,11
Exquisite Beryl Jewel,4019,2806,610,1830,3,7,22,70,-298,428,-4,156,86
Vial of Might,3521,2402,591,1182,2,9,25,55,-105,1350,-14,194,85
